In [8]:
import cv2
import numpy as np

img = cv2.imread('pictures/02.jpeg')
arr = np.linspace(img).reshape([5,5,])
print(arr)
print(img[0].size)
print(img[599])
median = cv2.medianBlur(img, 5)
compare = np.concatenate((img, median), axis=1) #side by side comparison

# cv2.imshow('img', compare)
# cv2.waitKey(0)
# cv2.destroyAllWindows

TypeError: _linspace_dispatcher() missing 1 required positional argument: 'stop'